objectives
- find the top hit for each contig in the mmseqs output, across all dbs
- determine what percentage of contigs were classified
- determine what percentage of reads were classified

find the top hit for each contig in the blast output

In [1]:
pkgs = [
    "Revise"
    "DataFrames",
    "StatsBase",
    "ProgressMeter"
]
import Pkg
# Pkg.activate(; temp=true)
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
┌ Warning: The active manifest file at `/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `import Pkg; Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types ~/.julia/juliaup/julia-1.9.3+0.x64.linux.gnu/share/julia/stdlib/v1.9/Pkg/src/manifest.jl:316
  No Changes to `/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/Project.toml`
  No Changes to `/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/Manifest.toml`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ /oak/stanford/scg/lab_mpsnyder/cjprybol/M

In [ ]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")
sample_directories = readdir(joinpath(data_dir, "SRA"), join=true)

In [ ]:
joint_lca_table = DataFrames.DataFrame()
easy_taxonomy_lca_reports = filter(x -> occursin("final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.", x) && occursin("_lca.tsv", x), readdir(joinpath(SRR_path, "mmseqs_easy_taxonomy"), join=true))
for lca_tsv in easy_taxonomy_lca_reports
    method = replace(replace(basename(lca_tsv), "final.contigs.fastg.gfa.fna." => ""), "_lca.tsv" => "")
    this_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(lca_tsv)
    this_lca_table[!, "SRR"] .= basename(SRR_path)
    this_lca_table[!, "SRR"] .= method
    append!(joint_lca_table, this_lca_table)
end
sort!(joint_lca_table, "contig_id")
joint_lca_table = joint_lca_table[map(x -> x in viral_tax_ids, joint_lca_table[!, "taxon_id"]), :]
